# HW 9: 自駕車安全 (資訊安全)

2020.11.27

## Tutorial

- [Lab9 Introduction by TA (10:51)](https://www.youtube.com/watch?v=wih8YGJrdb0&list=PLoZPctIb8M5RajPTWv9H2Nw9woLAdaSKQ)
- [Cybersecurity: Crash Course Computer Science #31 (12:29)](https://www.youtube.com/watch?v=bPVaOlJ6ln0)
- [The Internet: Encryption & Public Keys (06:39)](https://www.youtube.com/watch?v=ZghMPWGXexs)


## Objective

- Learn how to sniffer self network packets
- Understand what is cybersecurity


## Problem

在前面 `Lab7 自駕車間的溝通 (網路工程)`，我們可以透過網路與小鴨車交換資料  
讓其他使用者用 `remote_joystick`，便可以控制我們的小鴨車  

但這樣，我們的小鴨車是安全的嗎？  
會不會我們的控制資料可以被壞使用者一覽無遺

另外，平常在使用 `HTTP` 網站也是安全的嗎？是不是敏感資料也會被看光光

## Requirement

學習使用開源的 網路封包分析工具 `Wireshark`

- 去觀察 `Lab7 自駕車間的溝通 (網路工程)` 作業 (非挑戰) 所傳輸的 `TCP` 與 `UDP` 封包
  - 需觀察到送出馬達控制訊號的 `TCP` 封包內的資料 (payload)
  - 需觀察到送出車燈控制訊號的 `UDP` 封包內的資料 (payload)
  - 需觀察到小鴨車回覆馬達控制訊號的 `TCP` 封包內的資料 (payload)
- 使用 帳號密碼 登入 http://www.techpanda.org/ ，觀察 `HTTP` 封包內是否會曝露帳號密碼
  - 帳號: `admin@google.com` 密碼: `Password2010`
  - 該網站是該課程找來用於測試 `HTTP` 是否會曝露敏感資訊，其網站的架設與管理並非是該課程的教學團隊

## Steps 實作步驟

僅供參考，並非寫出作業的唯一方法  
只是其中一種方法

### Step1: 安裝 `Wireshark`，並開啟 `Wireshark`

可以到 [Download Wireshark](https://www.wireshark.org/download.html)  
根據作業系統選擇適合的安裝檔案

安裝完後，開啟 `Wireshark` 可以看到如下畫面

![](https://imgur.com/gG0pOsy.png)

### Step2: 選擇網路介面，開始擷取封包

在主頁面的 `Capture` 下可以看到多張網路介面卡 `interface`  
根據你正在使用的介面卡雙擊  
例如：該圖中，因為正在使用 `wifi` 所以選擇了 `wifi` 的介面卡

![](https://imgur.com/q3G8GLf.png)

`Wireshark` 就會開始去擷取在這張介面卡上的封包  
會看到有很多封包被一個一個顯示在列表上

![](https://imgur.com/IhLx2gW.png)

### Step3: 透過 filter expression 過濾封包

因為會有各種封包被擷取到，像是正在瀏覽網頁時的 `HTTP` 或 `TLS` 等等  
但我們只想要針對正在控制小鴨車的封包  
首先要思考控制小鴨車的封包有什麼特性？然後去將封包過濾出來  
因為 `Lab7 自駕車間的溝通 (網路工程)` 要求，我們的小鴨車固定都是監聽 `54760` 這個 port 埠號  
所以只要過濾出 **目標埠號為 `54760`** 的封包

嘗試在上方的 輸入框 輸入：`tcp.dstport == 54760`  
代表我們需要只把我們顯示出 `TCP` 的目標埠號 等於 `54760` 的封包  
記得按下 `Enter` 讓 `Wireshark` 應用該 filter expression 過濾表達式

![](https://imgur.com/bdHTQgw.png)

> 更詳細介紹可以看 [Wireshark UserGuide: The “Filter” Toolbar](https://www.wireshark.org/docs/wsug_html_chunked/ChUseFilterToolbarSection.html)

有可能會發現什麼封包都沒有顯示，那是因為我們還沒執行 `Lab7 自駕車間的溝通 (網路工程)` 的程式

### Step4: 用 `Lab7 hw` 控制小鴨車去觀察擷取到的封包

嘗試去控制小鴨車，會發現到 `Wireshark` 開始有擷取到相關的 `TCP` 封包

![](https://imgur.com/l246P0f.png)

### Step5: 修改 filter expression 找出相關的 `TCP` 與 `UDP` 封包

`tcp.dstport == 54760` 這條 filter expression 還不足以讓我們過濾出所有控制小鴨車相關的封包  
因此要修改 filter expression 去過濾出控制小鴨車相關的 `TCP` 與 `UDP` 封包

當一個控制小鴨車前進時，要可以看到

- 送出前進資料的 `TCP` 封包
- 送出前進燈號資料的 `UDP` 封包
- 小鴨車回覆的 `TCP` 封包

如下圖:

小鴨車為 `192.168.87.100` ，執行 `remote_joystick.py` 的電腦為 `192.168.87.101`

![](https://imgur.com/oTzHose.png)

- `No.779` 封包為 送出前進資料的 `TCP` 封包
- `No.780` 封包為 送出前進燈號資料的 `UDP` 封包
- `No.782` 封包為 小鴨車回覆的 `TCP` 封包

> 沒有連號是正常的，因為中間可能有其他諸如 `ICMP`, `HTTP` 封包

### Step6: 登入網站，並過濾出相關的 `HTTP` 封包

先開啟 `Wireshark` 的封包擷取，然後前往 http://www.techpanda.org/ 登入網站  
因為**該網站已經遭受 注入程式碼 攻擊，因此登入後很有可能看到不正常的頁面**  
但不影響我們去觀察登入帳密是否會曝露

透過在 filter expression 輸入 `http`，可以過濾出所有 `HTTP` 的封包  
找到用於登入的 `HTTP POST /index.php` 請求，嘗試觀察封包內是否可以看到帳號密碼

![](https://imgur.com/xd1EvHR.png)

## Notice

- deadline: 2020.12.04 before class

## Reference

- [Wireshark UserGuide: The “Filter” Toolbar](https://www.wireshark.org/docs/wsug_html_chunked/ChUseFilterToolbarSection.html)
- [Wireshark UserGuide: Building Display Filter Expressions](https://www.wireshark.org/docs/wsug_html_chunked/ChWorkBuildDisplayFilterSection.html)